## Soleil DiffAbs XRD:
* [Calibration](#calibration)
* [Integration](#integration)

## Calibration <a name="calibration"></a>

This tutorial is for the calibration of an Xpad_flat on the Soleil Diffabs Diffractometer.

The calibration data is a scan on the delta motor with a CeO2 calibrant.

In [2]:
%matplotlib notebook

# first the imports
from typing import Iterator, List, NamedTuple, Text, Tuple

import os
import functools

from collections import namedtuple
from math import pi

import numpy
import pylab
import pyFAI

import ipywidgets as widgets

from h5py import Dataset, File
from IPython.display import display
from numpy import ndarray
from pyFAI.gui import jupyter

# local import
from soleil import Angle, Calibrant, Calibration, CalibrationFunctions, CalibrationFrame, DatasetPathContains, DatasetPathWithAttribute, \
    Detector, Length, Parameter, Wavelength, \
    calibration, integrate, save_as_edf

In [3]:
# temporary until the ruch is ON
ROOT = os.path.join("/nfs", "ruche-diffabs", "diffabs-soleil", "com-diffabs", "2018", "Run2")

PUBLISHED = os.path.join("/nfs", "ruche-diffabs", "diffabs-soleil", "com-diffabs", "2018", "Run2B", "TPHercules")

In [4]:
# do a calibration with a well centered sample.

# TODO recuperer le point de départ à partir des ponis de référence.
# TODO ajouter le diagramme complet à la fin.
# TODO ajout du mask pour la calibration

json = os.path.join(PUBLISHED, "xrd", "calibration", "calibration.json")

distance = Length(0.644)
poni1 = Length(0.015639111692746385)
poni2 = Length(0.026262764736101073)
rot1_scale = -1
rot1_offset = Angle(-15.999503651711137)
rot2 = Angle(0.00038785472522193072)
rot3 = Angle(0)

functions = (
    CalibrationFunctions("dist", "poni1", "poni2",
                         "pi * (rot1_scale * delta + rot1_offset) / 180",
                         "rot2",
                         "rot3"),
    [Parameter[Length]("dist", distance, (distance-0.01, distance+0.01)),
     Parameter[Length]("poni1", poni1, (poni1, poni1)),
     Parameter[Length]("poni2", poni2, (poni2-0.01, poni2+0.01)),
     Parameter[float]("rot1_scale", rot1_scale, (rot1_scale-0.1, rot1_scale+0.1)),
     Parameter[Angle]("rot1_offset", rot1_offset, (rot1_offset-1, rot1_offset+1)),
     Parameter[float]("rot2", rot2, (rot2-0.1, rot2+0.1)),
     Parameter[Angle]("rot3", rot3, (rot3, rot3))]
)

def to_use(frame: CalibrationFrame) -> bool:
    return True if frame.idx in range(1,8) else False

params = Calibration(os.path.join(PUBLISHED, "xrd", "calibration"),
                     os.path.join(ROOT, "2018-03-21", "TPHercules_7.nxs"),
                     DatasetPathWithAttribute("interpretation", b"image"),
                     DatasetPathContains("scan_data/actuator_1_1"),
                     [1, 2, 3],
                     to_use,
                     Calibrant("CeO2"),
                     Detector("imxpads140"),
                     Wavelength(6.8877e-11),
                     functions
                    )

In [4]:
from functools import partial

def on_click_button(cmd, b):
    ! {cmd}
    
def my_cmd_button(cmd, description):
    # Click the button to open the data with silx
    button = widgets.Button(
        description=description,
        disabled=False,
        button_style='info', # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Click me',
        icon='check',
        layout = widgets.Layout(width='auto')
    )

    display(button)

    button.on_click(partial(on_click_button, cmd))

    return button

In [5]:
silx = my_cmd_button("silx view --use-opengl-plot " + params.filename,
                     'Silx View ' + os.path.basename(params.filename))

In [6]:
# save all the ref as images in order to do the calibration with
# pyFAI-calib[2].
# temporary until pyFAI-calib2 can read from NeXuS files.

cmds = save_as_edf(params)
for cmd, idx in zip(cmds, params.idxs):
    my_cmd_button(cmd, 'PyFAI-calib2 ' + os.path.basename(params.filename) + " " + str(idx))

In [7]:
# execute the calibration

calibration(json, params)

Empty refinement object:
GoniometerRefinement with 0 geometries labeled: .
Detector Imxpad S140	 PixelSize= 1.300e-04, 1.300e-04 m
Wavelength= 6.887700e-11m
SampleDetDist= 6.161367e-01m	PONI= -3.507805e-02, 1.358724e-01m	rot1=-0.044207  rot2= 0.078390  rot3= -0.000000 rad
DirectBeamDist= 618.639mm	Center: x=1254.830, y=102.827 pix	Tilt=5.155 deg  tiltPlanRotation= 60.638 deg
Detector Imxpad S140	 PixelSize= 1.300e-04, 1.300e-04 m
Wavelength= 6.887700e-11m
SampleDetDist= 6.073942e-01m	PONI= -1.526742e-01, -5.070196e-02m	rot1=-0.407438  rot2= 0.258864  rot3= 0.000001 rad
DirectBeamDist= 684.351mm	Center: x=1626.483, y=173.134 pix	Tilt=27.433 deg  tiltPlanRotation= 33.753 deg
Detector Imxpad S140	 PixelSize= 1.300e-04, 1.300e-04 m
Wavelength= 6.887700e-11m
SampleDetDist= 6.441112e-01m	PONI= 1.508603e-02, 3.603602e-02m	rot1=-0.308815  rot2= 0.000814  rot3= 0.000000 rad
DirectBeamDist= 676.095mm	Center: x=1857.856, y=120.280 pix	Tilt=17.694 deg  tiltPlanRotation= 0.153 deg
Filled refinement

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Cost function before refinement: 1.75270325262e-07
[  6.44000000e-01   1.56391117e-02   2.62627647e-02  -1.00000000e+00
  -1.59995037e+01   3.87854725e-04   0.00000000e+00]
     fun: 5.7851085934378249e-09
     jac: array([  8.43960068e-10,   6.16364682e-09,   2.69478566e-08,
        -2.68279676e-10,   1.31121097e-09,   1.54755819e-10,
        -1.74860126e-14,   0.00000000e+00])
 message: 'Optimization terminated successfully.'
    nfev: 154
     nit: 17
    njev: 17
  status: 0
 success: True
       x: array([  6.43805206e-01,   1.56391117e-02,   2.65135913e-02,
        -9.96093297e-01,  -1.59995061e+01,  -5.59136366e-05,
         0.00000000e+00])
Cost function after refinement: 5.78510859344e-09
GonioParam(dist=0.64380520621704995, poni1=0.015639111692746385, poni2=0.0265135913142076, rot1_scale=-0.99609329729560891, rot1_offset=-15.999506107308562, rot2=-5.591363662629065e-05, rot3=0.0)
maxdelta on: rot1_scale (3) -1 --> -0.996093297296




TPHercules_7.nxs_4
ControlPoints instance 

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [8]:
# plot the full diffractogram

def f(img: ndarray) -> ndarray:
    img = numpy.where(img > 20000, 0, img)
    return img

integrate(json, params, f, plot_calibrant=True)

<IPython.core.display.Javascript object>

## Integration <a name="integration"></a>

In [8]:
# integrate all the diffractogram

# let's take all the images
def to_use_sample(frame: CalibrationFrame) -> bool:
    return True


def mkCalibration(i: int) -> Calibration:
    return Calibration(os.path.join(PUBLISHED, "xrd", "calibration"),
                       os.path.join(ROOT, "2018-03-21", "TPHercules_{:d}.nxs".format(i)),
                       DatasetPathWithAttribute("interpretation", b"image"),
                       DatasetPathContains("scan_data/actuator_1_1"),
                       [1],
                       to_use_sample,
                       Calibrant("CeO2"),
                       Detector("imxpad_s140"),
                       Wavelength(6.8877e-11),
                       functions
                       )

tincrease = [mkCalibration(i) for i in range(113, 235)]
rtback = [mkCalibration(i) for i in range(243, 248)]
water = [mkCalibration(i) for i in range(253, 255)]
water2 = [mkCalibration(i) for i in range(256, 297)]
other = [mkCalibration(i) for i in [7, 300, 10, 11, 301, 299, 298]]

def f(img: ndarray) -> ndarray:
    img = numpy.where(img > 20000, 0, img)
    return img

samples = water + water2 + tincrease + rtback + other

for s in samples:
    print(s.filename)
    integrate(json, s, f, save=True)

/nfs/ruche-diffabs/diffabs-soleil/com-diffabs/2018/Run2/2018-03-21/TPHercules_253.nxs
/nfs/ruche-diffabs/diffabs-soleil/com-diffabs/2018/Run2/2018-03-21/TPHercules_254.nxs
/nfs/ruche-diffabs/diffabs-soleil/com-diffabs/2018/Run2/2018-03-21/TPHercules_256.nxs
/nfs/ruche-diffabs/diffabs-soleil/com-diffabs/2018/Run2/2018-03-21/TPHercules_257.nxs
/nfs/ruche-diffabs/diffabs-soleil/com-diffabs/2018/Run2/2018-03-21/TPHercules_258.nxs
/nfs/ruche-diffabs/diffabs-soleil/com-diffabs/2018/Run2/2018-03-21/TPHercules_259.nxs
/nfs/ruche-diffabs/diffabs-soleil/com-diffabs/2018/Run2/2018-03-21/TPHercules_260.nxs
/nfs/ruche-diffabs/diffabs-soleil/com-diffabs/2018/Run2/2018-03-21/TPHercules_261.nxs
/nfs/ruche-diffabs/diffabs-soleil/com-diffabs/2018/Run2/2018-03-21/TPHercules_262.nxs
/nfs/ruche-diffabs/diffabs-soleil/com-diffabs/2018/Run2/2018-03-21/TPHercules_263.nxs
/nfs/ruche-diffabs/diffabs-soleil/com-diffabs/2018/Run2/2018-03-21/TPHercules_264.nxs
/nfs/ruche-diffabs/diffabs-soleil/com-diffabs/2018/Run